In [32]:
import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector 
from cvzone.PlotModule import LivePlot
from scipy.spatial import distance as dist

# Video Capture from webcam
cap = cv2.VideoCapture(0)

#Detection of Number of Faces
detector = FaceMeshDetector(maxFaces=1)

#Live Plotof the Blink
plotY = LivePlot(640,360,[200,500], invert = True)

#Nodes for Eyes
idList = [7,33,133,144,145,153,154,155,157,158,159,160,161,246,373,374,380,382,384,385,386,387,388,390,398]

blinkCounterR = 0
counterR = 0
blinkCounterL = 0
counterL = 0

#Detection of the Face, Eye and Blink
while cap.isOpened():
    success, img = cap.read()
    img, faces = detector.findFaceMesh(img)
    
    #If there is a face in the camera;
    if faces:
        face = faces[0]
        for id in idList:
            cv2.circle(img, face[id], 5, (255,0,255))
            
        #Left Eye Data
        leftUp = face[159]
        leftUpR = face[158]
        leftUpL = face[160]
        leftDown = face[145]
        leftDownL = face[144]
        leftDownR = face[153]
        leftRight = face[133]
        leftLeft =  face[33]
        
        #Right Eye Data
        rightUp = face[386]
        rightUpR = face[387]
        rightUpL = face[385]
        rightDown = face[374]
        rightDownL = face[380]
        rightDownR = face[373]
        rightRight = face[263]
        rightLeft =  face[362]
        
        #EAR calculation for left Eye
        EARL = (dist.euclidean(leftDownL,leftUpL)+dist.euclidean(leftDownR,leftUpR))/(2*dist.euclidean(leftLeft,leftRight))
        
        #EAR calculation for right Eye
        EARR = (dist.euclidean(rightDownL,rightUpL)+dist.euclidean(rightDownR,rightUpR))/(2*dist.euclidean(rightLeft,rightRight))
        
        #Draw H and V Line for Left Eye
        cv2.line(img, leftUp, leftDown, (0, 200, 0), 3)
        cv2.line(img, leftRight, leftLeft, (0, 200, 0), 3)
        
        #Draw H and V Line for Right Eye
        cv2.line(img, rightUp, rightDown, (0, 200, 0), 3)
        cv2.line(img, rightRight, rightLeft, (0, 200, 0), 3)
            
        if EARL < 0.24 and counterL == 0:
            blinkCounterL += 1
            cv2.waitKey(300)
            counterL = 1
        if counterL != 0:
            counterL +=1
            if counterL > 10:
                if EARL > 0.27:
                    counterL = 0
        
        if EARR < 0.24 and counterR == 0:
            blinkCounterR += 1
            cv2.waitKey(300)
            counterR = 1
        if counterR != 0:
            counterR +=1
            if counterR > 10:
                if EARR > 0.27:
                    counterR = 0
        
        cvzone.putTextRect(img, f'BlinkCount: {blinkCounterL}', (50,100))
        cvzone.putTextRect(img, f'BlinkCount: {blinkCounterR}', (900,100))
        
        imgPlotL = plotY.update(EARL)
        imgPlotR = plotY.update(EARR)
        
        
    #cv2.imshow("Image", img)
    cv2.imshow("Image", img)
    cv2.waitKey(10)
    
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break
    
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()
ca